In [1]:
import os
from llama_index.llms import AzureOpenAI
from llama_index.embeddings import AzureOpenAIEmbedding
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
import logging
import sys
from dotenv import load_dotenv
from llama_index.llms import Ollama
import numpy as np
from trulens_eval import TruLlama, Feedback, Tru, feedback
from trulens_eval.tru_custom_app import instrument
tru = Tru()

load_dotenv('/Users/jeana/.env')

logging.basicConfig(
    stream=sys.stdout, level=logging.WARNING
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


### Input and Parameters

In [2]:
input_file_location = [r"/Users/jeana/Retrieval-Augmented-Generation/LlamaIndex/paul_graham_essay.txt"]

### Define Exceptions

In [3]:
class modelException(Exception):
    def __init__(self, invalid_value, allowed_values):
        self.invalid_value = invalid_value
        self.allowed_values = allowed_values
        message = f"Invalid value: {invalid_value}. Allowed values are: {', '.join(allowed_values)}"
        super().__init__(message)

### Define Evaluator with Feedback functions

In [4]:
def feedbacks():

    ##### INITIALZE FEEDBACK FUNCTION(S)#######

    # Initialize AzureOpenAI-based feedback function collection class:
    azopenai = feedback.AzureOpenAI(
                                    deployment_name=os.environ['OPENAI_DEPLOYMENT_NAME'],
                                    api_key = os.environ['OPENAI_API_KEY'],
                                    api_version=os.environ['OPENAI_DEPLOYMENT_VERSION'],
                                    azure_endpoint=os.environ['OPENAI_DEPLOYMENT_ENDPOINT'],
                                    # model = os.environ['OPENAI_MODEL_NAME']
                                    )

    # Question/answer relevance between overall question and answer.
    f_qa_relevance = Feedback(azopenai.relevance, name = "Answer Relevance").on_input_output()

    # Question/statement relevance between question and each context chunk.
    f_qs_relevance = Feedback(azopenai.qs_relevance, name = "Context Relevance").on_input().on(
        TruLlama.select_source_nodes().node.text
    ).aggregate(np.mean)

    # groundedness of output on the context
    groundedness = feedback.Groundedness(
                    # summarize_provider=azopenai, 
                    groundedness_provider=azopenai)
    f_groundedness = Feedback(groundedness.groundedness_measure, name = "Groundedness").on(TruLlama.select_source_nodes().node.text).on_output()
    
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance]
    
    return feedbacks


In [5]:
feedbacks = feedbacks()

Arguments ['api_key', 'api_version', 'azure_endpoint'] are ignored as `client` was provided.
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


### RAG from Scratch

In [6]:
class RAG():
    llm_list = ['llama2', 'gpt3.5Turbo']
    embedding_list = ['text-embedding-ada-002', 'sentence-transformers/all-mpnet-base-v2', "BAAI/bge-small-en-v1.5"]

    def __init__(self, llm_name, embedding_model, feedbacks):   
        if llm_name not in self.llm_list:
            raise modelException(llm_name, self.llm_list)
        if embedding_model not in self.embedding_list:
            raise modelException(embedding_model, self.embedding_list)        

        ## Getting the LLM for prediction
        if llm_name == 'gpt3.5Turbo':
            self.llm = AzureOpenAI(
                    # model= os.environ['OPENAI_MODEL_NAME'],
                    model = llm_name,
                    deployment_name= os.environ['OPENAI_DEPLOYMENT_NAME'],
                    api_key=os.environ['OPENAI_API_KEY'],
                    azure_endpoint=os.environ['OPENAI_DEPLOYMENT_ENDPOINT'],
                    api_version=os.environ['OPENAI_DEPLOYMENT_VERSION'],
                )
        elif llm_name == 'llama2':
            self.llm = Ollama(model="llama2")
        
        ## Gettting the embedding model
        if embedding_model == 'text-embedding-ada-002':
            self.embedding_model = AzureOpenAIEmbedding(
                    # model=os.environ['OPENAI_EMBEDDING_MODEL_NAME'],
                    model=embedding_model,
                    deployment_name=os.environ['OPENAI_EMBEDDING_DEPLOYMENT_NAME'],
                    api_key=os.environ['OPENAI_API_KEY'],
                    azure_endpoint=os.environ['OPENAI_DEPLOYMENT_ENDPOINT'],
                    api_version=os.environ['OPENAI_DEPLOYMENT_VERSION'],
                )
        elif embedding_model in ['sentence-transformers/all-mpnet-base-v2', "BAAI/bge-small-en-v1.5"]:
            self.embedding_model = HuggingFaceEmbeddings(
                    model_name=embedding_model)
        
        #Set service context, documents and Index
        self.service_context = ServiceContext.from_defaults(embed_model=self.embedding_model, llm = self.llm)

        self.documents = SimpleDirectoryReader(
                    input_files=[r"/Users/jeana/Retrieval-Augmented-Generation/LlamaIndex/paul_graham_essay.txt"] #or just indicate the fullpath of the folder containing the data
                                ).load_data()
        self.index = VectorStoreIndex.from_documents(self.documents, service_context=self.service_context)

    def query(self, query: str) -> str:
        ### INSTRUMENT CHAIN FOR LOGGING WITH TRULENS

        query_engine = self.index.as_query_engine()

        tru_query_engine_recorder = TruLlama(query_engine,
                app_id='LlamaIndex_App1',
                feedbacks=feedbacks)

        with tru_query_engine_recorder as recorder:
            answer = query_engine.query(query)
            print(answer.get_formatted_sources())
            print("query was:", query)
            print("answer was:", answer)

In [7]:
llm_name =  'llama2' #gpt3.5Turbo , llama2
embedding_model = 'sentence-transformers/all-mpnet-base-v2' # text-embedding-ada-002 , sentence-transformers/all-mpnet-base-v2, local
rag = RAG(llm_name, embedding_model, feedbacks)

In [ ]:
rag.query('What is most interesting about this essay?')

> Source (Doc id: 8bb6e429-2e73-45aa-b52d-46b1f069d52c): I remember taking the boys to the coast on a sunny day in 2015 and figuring out how to deal with ...

> Source (Doc id: 749ce9a8-3351-4832-a4f8-35e7cd742a26): One of the most conspicuous patterns I've noticed in my life is how well it has worked, for me at...
query was: What is most interesting about this essay?
answer was: The most interesting aspect of this essay is the author's reflection on how his choices and experiences have been shaped by his desire to work on things that are not prestigious, but instead genuinely interest him. He notes that when he finds himself drawn to something despite its lack of prestige, it's a sign that there's something real to be discovered there, and that he has the right kind of motives. He also shares how he and his friends have started their own investment firm, after becoming frustrated with the slow decision-making of venture capital firms.

Overall, the essay provides a personal and intros

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Feedback function `groundedness_measure` was renamed to `groundedness_measure_with_cot_reasons`. The new functionality of `groundedness_measure` function will no longer emit reasons as a lower cost option. It may have reduced accuracy due to not using Chain of Thought reasoning in the scoring.


In [11]:
#View results in notebook
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness:::full_doc_score,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,LlamaIndex_App1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_428bceed859a74951403887e04431b27,"""What is most interesting about this essay?""","""The most interesting aspect of this essay is ...",-,"{""record_id"": ""record_hash_428bceed859a7495140...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-26T16:56:58.102181"", ""...",2023-12-26T16:57:29.004522,0.8,0.2,0.0,[{'args': {'prompt': 'What is most interesting...,[{'args': {'question': 'What is most interesti...,[{'args': {'source': 'I remember taking the bo...,30,0,0.0


In [12]:
#view results in dashboard
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.20.13:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>